# Tutorial: Human-in-the-Loop with Haystack Agents

- **Level**: Intermediate
- **Time to complete**: 20 minutes
- **Components Used**: [`OpenAIChatGenerator`](https://docs.haystack.deepset.ai/docs/openaichatgenerator)
- **Experimental Components Used**: [`Agent`](https://docs.haystack.deepset.ai/docs/agent)
- **Prerequisites**: You need an [OpenAI API Key](https://platform.openai.com/api-keys)
- **Goal**: After completing this tutorial, you'll have learned how to implement human-in-the-loop workflows in Haystack agents using confirmation strategies, create custom confirmation policies, and control tool execution approval flows.

## Overview
This tutorial introduces how to use **confirmation strategies** to create **human-in-the-loop interactions** in Haystack's `Agent` component. 

**Why is this useful?** Imagine an AI agent that can access your bank account, send emails, or make purchases. You probably want human approval before it executes sensitive operations. Confirmation strategies let you define exactly when the agent should ask for permission.

## How Confirmation Strategies Work
1. User sends a query to the agent
2. Agent determines which tool(s) to use
3. Before executing, the confirmation policy checks if approval is needed
4. If needed, the UI prompts the user for confirmation
5. Based on user response (confirm/reject/modify), the agent proceeds or adjusts
6. Agent returns the final answer to the user

## Preparing the Environment

First, let's install required packages:

In [2]:
%%bash

pip install -q haystack-experimental

### Enter API Keys

Enter API keys required for this tutorial.

In [ ]:
from getpass import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key:")

## Setup and Imports

We begin by importing Haystack components, UI helpers, and the experimental Agent API.

In [4]:
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage
from haystack.tools import create_tool_from_function
from rich.console import Console

from haystack_experimental.components.agents.agent import Agent
from haystack_experimental.components.agents.human_in_the_loop import (
    AlwaysAskPolicy,
    AskOncePolicy,
    BlockingConfirmationStrategy,
    NeverAskPolicy,
    RichConsoleUI,
    SimpleConsoleUI,
)

## Define Example Tools

We create three simple tools for demonstration: `addition`, `get_bank_balance`, and `get_phone_number`.

In [5]:
def addition(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b


addition_tool = create_tool_from_function(function=addition, name="addition", description="Add two floats together.")


def get_bank_balance(account_id: str) -> str:
    """Simulate fetching a bank balance."""
    return f"Balance for account {account_id} is $1,234.56"


balance_tool = create_tool_from_function(
    function=get_bank_balance, name="get_bank_balance", description="Get the bank balance for a given account ID."
)


def get_phone_number(name: str) -> str:
    """Simulate fetching a phone number."""
    return f"The phone number for {name} is (123) 456-7890"


phone_tool = create_tool_from_function(
    function=get_phone_number, name="get_phone_number", description="Get the phone number for a given name."
)

## Define the Agent

We instantiate an `Agent` with multiple tools and assign each tool a **confirmation strategy**.

In [6]:
cons = Console()

agent = Agent(
    chat_generator=OpenAIChatGenerator(model="gpt-4.1-mini"),
    tools=[balance_tool, addition_tool, phone_tool],
    system_prompt="You are a helpful financial assistant. Use the provided tools to answer user questions.",
    confirmation_strategies={
        balance_tool.name: BlockingConfirmationStrategy(
            confirmation_policy=AlwaysAskPolicy(), confirmation_ui=RichConsoleUI(console=cons)
        ),
        addition_tool.name: BlockingConfirmationStrategy(
            confirmation_policy=NeverAskPolicy(), confirmation_ui=RichConsoleUI(console=cons)
        ),
        phone_tool.name: BlockingConfirmationStrategy(
            confirmation_policy=AskOncePolicy(), confirmation_ui=RichConsoleUI(console=cons)
        ),
    },
)

### Explanation

Each `BlockingConfirmationStrategy` defines when and how the agent asks for confirmation before executing a tool:

* **`AlwaysAskPolicy`** – Always asks for approval before running.
* **`NeverAskPolicy`** – Executes automatically without user confirmation.
* **`AskOncePolicy`** – Asks once per tool, remembers approval for future runs.

The UI can be either:

* `RichConsoleUI` for colorized and more aesthetic prompts.
* `SimpleConsoleUI` for basic prompts.


Both UIs will present three options:

- **y**: Proceed with the tool execution as planned
- **n**: Cancel the tool execution; agent will try alternative approaches
- **m**: Edit the tool parameters before execution

**NOTE**: Custom UIs and Policies can also be implemented by following the respective `ConfirmationUI` and `ConfirmationPolicy` protocols. This allows full flexibility for domain-specific workflows.

## Run the Agent

Now we can run the agent with different confirmation behaviors.

Try different confirmation strategies and explore the available options returned by the UIs: "y", "n", "m". This lets you test how the agent reacts to different human feedback scenarios.

### Always Ask (Rich UI)

In [7]:
result = agent.run([ChatMessage.from_user("What's the balance of account 56789?")])
cons.print(f"\n[bold green]Agent Result:[/bold green] {result['last_message'].text}")

╭─ 🔧 Tool Execution Request ─────────────────────────────────────────────────────────────────────────────────────╮
│ Tool: get_bank_balance                                                                                          │
│ Description: Get the bank balance for a given account ID.                                                       │
│                                                                                                                 │
│ Arguments:                                                                                                      │
│ account_id: 56789                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Your choice [y/n/m] (y):

 y


Agent Result: The balance of account 56789 is $1,234.56.

### Never Ask

In [8]:
result = agent.run([ChatMessage.from_user("What is 5.5 + 3.2?")])
cons.print(f"\n[bold green]Agent Result:[/bold green] {result['last_message'].text}")

Agent Result: 5.5 + 3.2 equals 8.7.

### Ask Once

In [9]:
result = agent.run([ChatMessage.from_user("What is the phone number of Alice?")])
cons.print(f"\n[bold green]Agent Result:[/bold green] {result['last_message'].text}")

╭─ 🔧 Tool Execution Request ─────────────────────────────────────────────────────────────────────────────────────╮
│ Tool: get_phone_number                                                                                          │
│ Description: Get the phone number for a given name.                                                             │
│                                                                                                                 │
│ Arguments:                                                                                                      │
│ name: Alice                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Your choice [y/n/m] (y):

 y


Agent Result: The phone number of Alice is (123) 456-7890.

### Ask Once: Cached Confirmation

If you answered "y" (yes) to the previous phone number request, the agent will not ask for confirmation again for subsequent requests to the same tool called with the same parameters.

In [10]:
result = agent.run([ChatMessage.from_user("What is the phone number of Alice?")])
cons.print(f"\n[bold green]Agent Result:[/bold green] {result['last_message'].text}")

Agent Result: The phone number of Alice is (123) 456-7890.

## Run an Agent with a Custom Policy

Now let's create a custom confirmation policy that asks for confirmation only when certain conditions are met. For example, we can create a policy that asks for confirmation when the tool operation involves expenses above a certain threshold.

### Create a Custom Budget Based Policy

Here we define a `BudgetBasedPolicy` that asks for confirmation if the cost exceeds a defined threshold.

In [11]:
from haystack_experimental.components.agents.human_in_the_loop import ConfirmationPolicy
from typing import Any


class BudgetBasedPolicy(ConfirmationPolicy):
    """Ask for confirmation when operations exceed a cost threshold."""

    def __init__(self, cost_threshold: float = 10.0):
        self.cost_threshold = cost_threshold

    def should_ask(self, tool_name: str, tool_description: str, tool_params: dict[str, Any]) -> bool:
        """Ask if the operation cost exceeds the threshold."""
        # Check for cost-related parameters
        cost = tool_params.get("cost", 0.0)
        amount = tool_params.get("amount", 0.0)
        price = tool_params.get("price", 0.0)

        return max(cost, amount, price) > self.cost_threshold

### Define an Agent with Expense Tool

Now we define an `expense` tool that simulates submitting an expense report and use the `BudgetBasedPolicy` for confirmation.

In [20]:
from haystack.components.generators.utils import print_streaming_chunk

In [22]:
def expense(cost: float, description: str) -> float:
    """Submit an expense report that has a `cost` and `description`"""
    # This is where we would add a real submission request
    return "Expense report submitted successfully!"


expense_tool = create_tool_from_function(
    function=expense, name="expense", description="Submit an expense report that has a `cost` and `description`"
)

cons = Console()

agent = Agent(
    chat_generator=OpenAIChatGenerator(model="gpt-4.1"),
    tools=[expense_tool],
    system_prompt=(
        "You are a helpful financial assistant that can submit expense reports for users. "
        "Use the `expense` tool which only requires a cost amount and short description of the expense (e.g. 'busines lunch'). "
        "Only respond with whether the report was submitted successfully. "
    ),
    confirmation_strategies={
        expense_tool.name: BlockingConfirmationStrategy(
            confirmation_policy=BudgetBasedPolicy(cost_threshold=100.0), confirmation_ui=RichConsoleUI(console=cons)
        )
    },
)

### Run the Agent

Now let's submit an expense report that's below the threshold

In [23]:
result = agent.run([ChatMessage.from_user("Submit an expense report for business lunch that cost $10")])
cons.print(f"\n[bold green]Agent Result:[/bold green] {result['last_message'].text}")

Agent Result: Expense report submitted successfully!

Next let's submit a request that's above the threshold

In [25]:
result = agent.run([ChatMessage.from_user("Submit an expense report for business travel that cost $200")])
cons.print(f"\n[bold green]Agent Result:[/bold green] {result['last_message'].text}")

╭─ 🔧 Tool Execution Request ─────────────────────────────────────────────────────────────────────────────────────╮
│ Tool: expense                                                                                                   │
│ Description: Submit an expense report that has a `cost` and `description`                                       │
│                                                                                                                 │
│ Arguments:                                                                                                      │
│ cost: 200                                                                                                       │
│ description: business travel                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Your choice [y/n/m] (y):

 n


Feedback message (optional) ():

 Denied by manager


Agent Result: The expense report submission was denied by the manager.

## What's next

🎉 Congratulations! You've just built a Haystack Agent that incorporates human-in-the-loop workflows using confirmation strategies.

Curious to keep exploring? Here are a few great next steps:

* [Creating a Multi-Agent System with Haystack](https://haystack.deepset.ai/tutorials/45_creating_a_multi_agent_system)
* [Introduction to Multimodal Text Generation](https://haystack.deepset.ai/cookbook/multimodal_intro)
* [AI Guardrails: Content Moderation and Safety with Open Language Models](https://haystack.deepset.ai/cookbook/safety_moderation_open_lms)

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates) or [join Haystack discord community](https://discord.gg/Dr63fr9NDS).

(*Notebook by [Sebastian Husch Lee](https://github.com/sjrl)*)